In [32]:
import pandas as pd
#!pip install neattext
import pandas as pd
import neattext.functions as nfx
import matplotlib.pyplot as plt

df = pd.read_csv('udemy_courses.csv', encoding='latin-1')
# Clean Text:stopwords,special charac
df['clean_course_title'] = df['course_title'].apply(nfx.remove_stopwords)

In [33]:
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,clean_course_title
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,Ultimate Investment Banking Course
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,Complete GST Course & Certification - Grow Pra...
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5 hours,2016-12-19T19:26:30Z,Business Finance,Financial Modeling Business Analysts Consultants
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3 hours,2017-05-30T20:07:24Z,Business Finance,Beginner Pro - Financial Analysis Excel 2017
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2 hours,2016-12-13T14:57:18Z,Business Finance,Maximize Profits Trading Options


In [64]:
#df[df['course_title'] != df['clean_course_title']][['course_title', 'clean_course_title']]


In [3]:
df.columns

Index(['course_id', 'course_title', 'url', 'is_paid', 'price',
       'num_subscribers', 'num_reviews', 'num_lectures', 'level',
       'content_duration', 'published_timestamp', 'subject',
       'clean_course_title'],
      dtype='object')

## retirando a parte comum da URL, traços e barras

In [ ]:
titulos = list(df['course_title'])
mapping = ['.',':','*','(',')','/','!','?','!','&',',','-','[',']','+','|']
for m in mapping:
    titulos = [t.replace(m, '') for t in titulos]

urls = list(df['url'])
urls = [u.replace('https://www.udemy.com/', '') for u in urls]

In [9]:
df.url = df.url.str.replace('https://www.udemy.com/','')
df.url = df.url.str.replace('-',' ')
df.url = df.url.str.replace('/','')

## função para criar o vocabulário de palavras distintas das colunas URL e COURSE_TITLE

In [10]:
def Criavocab (string):
    for t in string:
        palavras = t.lower().replace('.','').split()
        for p in palavras:
            if p not in vocabulario:
                vocabulario.append(p)

## Popula a variável "textos" com todas as frases (títulos e url)

In [11]:
vocabulario = []

textos_title = list(df['course_title'])
Criavocab(textos_title)
textos_url = list(df['url'])
Criavocab(textos_url)

print(len(set(vocabulario)))

5551


In [12]:
print (vocabulario[:20])

['ultimate', 'investment', 'banking', 'course', 'complete', 'gst', '&', 'certification', '-', 'grow', 'your', 'ca', 'practice', 'financial', 'modeling', 'for', 'business', 'analysts', 'and', 'consultants']


In [14]:
import nltk
nltk.download('stopwords')

from stop_words import get_stop_words
from nltk.corpus import stopwords

stop_words = list(get_stop_words('en'))         #Have around 900 stopwords
nltk_words = list(stopwords.words('english'))   #Have around 150 stopwords
stop_words.extend(nltk_words)

sentence = "i told her not to come home"   #Your sentence
tokens = nltk.word_tokenize(sentence)
output = []

for words in tokens:
    if not words in stop_words:
        output.append(words)

print (output)

['told', 'come', 'home']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Cria dicionário das palavras e sua quantidade


In [ ]:
qd_palavras = dict()
frases = textos

for frase in frases:
    for x in frase.split():
        #print (x)
        if x in qd_palavras:
            qd_palavras[x] += 1
        else:
            qd_palavras[x] = 1
        
print (len(qd_palavras))

In [ ]:
#max(qd_palavras, key=qd_palavras.get)
#qd_palavras[max(qd_palavras, key=qd_palavras.get)]


In [ ]:
#dict(sorted(qd_palavras.items(), key=lambda item: item[1], reverse = True))

In [ ]:
# agora é necessário representar cada frase usando o vocabulário
amostras = []
for t in textos:
    amostra = []
    for p in vocabulario:
        if p in t.lower().replace('.','').split():
            amostra.append(1)
        else:
            amostra.append(0)
            
    amostras.append(amostra)


In [ ]:
import numpy as np
amostras = np.array(amostras)

In [ ]:
for p, a in zip(vocabulario, amostras[0]):
    print(p, a)

In [ ]:
# vamos separar os dados em 33% pra teste e com semente aleatória fixa em 42

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(amostras, df['subject'], test_size=0.33, random_state=42)


In [ ]:
# temos 3 modelos, vamos avaliar diferentes cenários e observar os resultados

from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.metrics import accuracy_score

model = BernoulliNB()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))

In [ ]:
# vamos explorar o que o modelo armazenou de informação

print(model.classes_)
prob = model.feature_log_prob_

for idx, p in enumerate(vocabulario):
    print(p,'\t\t',prob[0][idx],prob[1][idx],prob[2][idx],prob[3][idx])

In [ ]:
# como testar com uma amostra nova

amostra_nova = 'bank'
amostra = []
for p in vocabulario:
    if p in amostra_nova.lower().replace('.','').split():
        amostra.append(1)
    else:
        amostra.append(0)
            
print(model.predict([amostra]))
print(model.predict_proba([amostra]))